## <6-2 개체명 인식 모델 학습하기>  

개체명 인식 데이터 전처리와 모델의 학습 과정을 살펴보겠습니다.

### 개체명을 인식하는 모델 만들기  
---
<**1단계**> **코랩 노트북 초기화하기**   
이번 실습은 웹 브라우저에서 다음 주소(bit.ly/3oMA8L1)에 접속하면 코랩 환경에서 수행할 수 있습니다. 이전 실습과 마찬가지로 코랩에 접속한 후, `[내 드라이브에 복사]`와 런타임 유형에서 하드웨어 가속을 `[GPU]`로 선택합니다. 

<**2단계**> **각종 설정하기**  
1단계 코랩 노트북 초기화 과정에서 하드웨어 가속기를 TPU로 선택했다면 다음 코드를 실행하고, GPU를 선택했다면 실행하지 않습니다.

In [ ]:
#코드 6-1 TPU 관련 패키지 설치
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

다음 코드를 실행해 TPU 이외에 의존성 있는 패키지를 설치하고 자신의 구글 드라이브를 코랩 노트북과 연결합니다.

In [ ]:
#코드 6-2 의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 578 kB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 806 kB 30.6 MB/s 
     |████████████████████████████████| 2.8 MB 51.7 MB/s 
     |████████████████████████████████| 636 kB 70.6 MB/s 
     |████████████████████████████████| 136 kB 64.4 MB/s 
     |████████████████████████████████| 829 kB 63.6 MB/s 
     |████████████████████████████████| 398 kB 63.9 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 895 kB 60.1 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 1.1 MB 48.2 MB/s 
     |████████████████████████████████| 96 kB 6.1 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 68.9 MB/s 
     |████████████████████████████████| 271 kB 56.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [ ]:
#코드 6-3 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이번 실습에서는 이준범 님이 공개한 `kcbert-base` 모델을 실습 데이터로 파인튜닝해 볼 예정입니다.  
실습 데이터로는 한국해양대학교 자연언어처리연구실에서 공개한 데이터와 자체 제작한 데이터(`ner`)를 합쳐 사용합니다.  
다음 코드를 실행하면 관련 설정을 할 수 있습니다. 개체명 인식을 위한 환경 설정은 문서 분류와 본질적으로 다르지 않습니다. 이 코드에서 설정하는 각종 인자에 관한 설명은 <4-2>절을 참고 바랍니다.

In [ ]:
#코드 6-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.ner import NERTrainArguments
args = NERTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="ner",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

이어서 다음 코드를 실행해 랜덤 시드를 고정하고 로거를 설정합니다.

In [ ]:
#코드 6-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [ ]:
#코드 6-6 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


<**3단계**> **말뭉치 내려받기**  
다음 코드를 실행하면 `corpus_name(ner)`애 해당하는 말뭉치를 내려받습니다.

In [ ]:
#코드 6-7 말뭉치 다운로드
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 17.9M/17.9M [00:00<00:00, 34.6MB/s]
Downloading: 100%|██████████| 1.13M/1.13M [00:00<00:00, 40.6MB/s]


<**4단계**> **토크나이저 준비하기**  
다음 코드를 실행해 `kcbert-base` 모델이 사용하는 토크나이저를 선언합니다.

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

<**5단계**> **데이터 전처리하기**  
다음 코드를 수행하면 데이터셋을 만들 수 있습니다. `NERCorpus` 클래스는 텍스트 파일을 '원본 문장 + 개체명 태그를 레이블한 문장' 형태로 읽어들입니다. `NERCorpus`는 `NERDataset`이 요구하면 원본 문장과 레이블한 문장을 `NERDataset`에 제공합니다.

In [ ]:
#코드 6-9 학습 데이터셋 구축
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
corpus = NERCorpus(args)
train_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/ner/train.txt
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 이어 옆으로 움직여 김일성의 오른쪽에서 한 차례씩 두 번 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.
INFO:ratsnlp:target: 이어 옆으로 움직여 <김일성:PER>의 오른쪽에서 <한 차례:NOH>씩 <두 번:NOH> 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.

INFO:ratsnlp:tokens: [CLS] 이어 옆 ##으로 움직 ##여 김일성 ##의 오른 ##쪽에서 한 차례 ##씩 두 번 상 ##체를 굽 ##혀 조문 ##했 ##으며 이 ##윽 ##고 안 ##경을 벗고 손 ##수 ##건으로 눈 ##주 ##위를 닦 ##기도 했다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: [CLS] O O O O O B-PER O O O B-NOH I-NOH O B-NOH I-NOH O O O O O O O O O O O O O O O O O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:features:

`NERDataset` 클래스는 `NERCorpus`와 코드 6-8에서 선언해 둔 토크나이저를 품고 있습니다.  
`NERDataset`은 `NERCorpus`가 넘겨 준 데이터(원본 문장, 레이블한 문장)를 모델이 학습할 수 있는 형태로 가공합니다. 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블한 문장을 모델이 읽어들일 수 있는 포맷(`NERFeatures`)으로 바꿔 주는 역할을 합니다.

`NERFeatures`라는 자료형에는 네 가지 정보가 있습니다.  
첫 번째는 `input_ids`입니다. 인덱스로 변환된 토큰 시퀀스입니다. 두 번째는 `attention_mask`로 해당 토큰이 패딩 토큰인지(`0`), 아닌지(`1`)를 나타냅니다. `token_type_ids`는 세그먼트, `label_ids`은 정수로 바뀐 레이블 시퀀스입니다. 

`NERFeatures` 각 구성 요소의 자료형은 다음과 같습니다.  

- **input_ids**: List[int]
- **attention_mask**: List[int]
- **token_type_ids**: List[int]
- **label_ids**: List[int]

그러면 말뭉치의 3번 데이터를 살펴봅시다.   
다음과 같습니다. 여기서 RS는 구분자이며, 이해를 돕고자 개체명에 해당하는 부분을 별도 색상으로 표시했습니다.  

> ㅡ효진 역의 김환희(14)가 특히 인상적이었다.$_{RS}$ㅡ**<효진:PER>** 역의 **<김환희:PER>**(**<14:NOH>**)가 특히 인상적이었다.

이 내용에 대응하는 `NERCorpus` 데이터는 다음과 같습니다.  
- **text**: ㅡ효진 역의 김환희(14)가 특히 인상적이었다.  
- **label**: ㅡ<효진:PER>역의 <김환희:PER>(<14:NOH>)가 특히 인상적이었다.

`NERDataset`은 우선 원본 문장(`text`)을 토큰화합니다.  
그 결과는 다음과 같습니다. 원본 문장의 토큰 시퀀스 앞뒤에 각각 `[CLS]`와`[SEP]`를 붙이고 코드 6-3의 `max_seq_length`(=64)가 되도록 패딩 토큰을 뒷부분에 추가했습니다.

이후 `NERDataset`은 레이블한 문장(`label`)을 바탕으로 레이블 시퀀스를 만듭니다. 문장 앞뒤에 각각 `[CLS]`와 `[SEP]`를 붙이고 코드 6-3의 `max_seq_length`(=64)가 되도록 패딩 토큰을 추가하는 원칙은 이전과 같습니다.   
레이블한 문장을 보면 개체명은 총 3개가 있는데요, `<효진:PER>`, `<김환희:PER>`, `<14:NOH>`이 바로 그것입니다.

**[CLS] [UNK] 효 ##진 역 ##의 김 ##환 ##희 ( 14 ) 가 특히 인상 ##적이 ##었다  .   [SEP]**  
**[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]**  
**[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]**  
**[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]**  
**[PAD]**

`PER`(인명)로 레이블링된 `효진`의 범위는 위의 토큰화 결과에서 세 번째 토큰(`효`)부터 네 번째 토큰(`##진`)인 걸 확인할 수 있습니다. 이에 다음 레이블 시퀀스의 세 번째 토큰과 네 번째 토큰이 `PER`(인명)이 되도록 합니다.  
단, 여기에서 `B-`는 해당 태그의 시작(begin), `I-`는 해당 태그의 시작이 아님(inside)을 의미합니다.

`PER`(인명)로 레이블링된 `김환희`는 일곱 번째 토큰(`김`)부터 아홉 번째 토큰(`##희`)입니다. 이에 레이블 시퀀스의 일곱 번째 토큰과 아홉 번째 토큰이 `PER`(인명)이 되도록 합니다. 마찬가지로 `14`의 경우 `labels`의 열한 번째 토큰이 `NOH`(기타 수량 표현)가 되도록 만들었습니다. 한편 다음 레이블 시퀀스에서 `0`은 outside의 약자로, 개체명이 아님을 의미합니다.

< 레이블 시퀀스 >   

**[CLS] 0 B-PER I-PER 0 0 B-PER I-PER I-PER 0 B-NOH 0 0 0 0 0 0 0 [SEP] [PAD] [PAD]**  
**[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]**  
**[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]**  
**[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]**  
**[PAD]**


코랩에서 코드 6-9를 실행한 뒤 `train_dataset[3]`을 입력하면 다음과 같은 결과를 확인할 수 있습니다. `NERCorpus`가 넘겨준 3번 데이터가 최종적으로 `NERFeatures`라는 3번 인스턴스로 변환된 것입니다. 이처럼 `NERDataset`이 가지고 있는 모든 인스턴스는 인덱스로 접근 가능합니다.

In [ ]:
train_dataset[3]

NERFeatures(input_ids=[2, 1, 3476, 4153, 2270, 4042, 420, 4185, 4346, 11, 11524, 12, 197, 9250, 11662, 8805, 8217, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label_ids=[0, 4, 5, 15, 4, 4, 5, 15, 15, 4, 6, 4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

`train_dataset[3].input_ids`는 원본 문장을 토큰화한 뒤 인덱싱을 수행한 결과이며 `train_dataset[3].attention_mask`는 해당 위치 토큰이 패딩인지(`0`) 아닌지(`1`)를 나타냅니다. `train_dataset[3].token_type_ids`는 세그먼트 정보로, 모두 `0`으로 넣습니다. 마지막으로 `train_dataset[3].label_ids`는 토큰별 각 개체명 태그(`B-PER, I-PER` 등)를 정수로 바꾼 결과입니다.

다음 코드를 실행하면 학습, 평가용 데이터 로더를 만들 수 있습니다.   
데이터셋을 제외한 `sampler`, `collate_fn` 등 모든 인자는 이전 장에서 실습한 것과 동일합니다.

In [ ]:
#코드 6-10 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [ ]:
#코드 6-11 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:loading val data... LOOKING AT /content/Korpora/ner/val.txt
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다
INFO:ratsnlp:target: 결국 초연은 <4년 반:DUR>이 지난 후에 <드레스덴:LOC>에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다

INFO:ratsnlp:tokens: [CLS] 결국 초 ##연 ##은 4년 반 ##이 지난 후에 드 ##레스 ##덴 ##에서 연 ##주 ##되었고 재 ##연 ##도 이루어 ##졌지 ##만 , 이후에 그대로 방치 ##되고 말았 ##다 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: [CLS] O O O O B-DUR I-DUR O O O B-LOC I-LOC I-LOC O O O O O O O O O O O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INF

<**6단계**> **모델 불러오기**  
다음 코드를 수행해 `kcbert-base`로 모델을 초기화합니다. 물론 허깅페이스 `transformers`에 등록된 모델이라면 다른 모델 역시 사용할 수 있습니다. `BertForTokenClassification`은 프리트레인을 마친 BERT 모델 위에 <6-1>절에서 설명한 개체명 인식을 위한 태스크 모듈이 덧붙여진 형태의 모델 클래스입니다. 

In [ ]:
#코드 6-12 모델 초기화
from transformers import BertConfig, BertForTokenClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForTokenClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the

<**7단계**> **모델 학습시키기**  
다음 코드를 실행하면 개체명 인식을 위한 태스크(`task`)를 정의할 수 있습니다. 모델은 코드 6-12에서 준비한 모델 클래스를 사용하고 옵티마이저로는 아담(Adam), 러닝 레이트 스케줄러는 `ExponentialLR`을 사용합니다.

In [ ]:
#코드 6-13 태스크 정의
from ratsnlp.nlpbook.ner import NERTask
task = NERTask(model, args)

다음 코드를 실행하면 트레이너를 정의할 수 있습니다. 이어 트레이너의 `fit()` 함수를 호출하면 학습을 시작합니다.

In [ ]:
#코드 6-14 트레이너 정의
trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-ner exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
#코드 6-15 학습 개시
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | BertForTokenClassification | 108 M 
-----------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.389   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

지금까지 우리는 시퀀스 레이블링의 대표 과제인 개체명 인식 모델 학습을 실습해 보았습니다.  
문서 분류(4장), 문장 쌍 분류(5장)는 입력 문서가 속하는 범주가 무엇인지 분류하지만, 시퀀스 레이블링은 개별 입력 토큰 각각의 범주를 가려낸다는 점에서 차이가 있습니다. 다시 말해 범주 수가 $m$개이고 입력 토큰이 $n$개일 때 문서 분류, 문장 쌍 분류 모델 출력은 $m$차원의 확률 벡터 1개입니다. 반면 시퀀스 레이블링은 $m$차원 확률 벡터가 $n$개 만들어집니다.

세 과제의 차이를 표로 나타내면 다음과 같습니다.  
어떤 모델을 사용할지는 수행하려는 태스크의 속성을 면밀히 파악한 뒤 결정해야 합니다.

<center><표 - 문서 분류, 문장 쌍 분류, 시퀀스 레이블링의 차이></center>



|방법론|입력|출력|대표 과제|
|:---:|:---:|:---:|:---:|
|문서 분류|문서(혹은 문장) 1개|한 문서가 속하는 범주에 대한 확률|감성 분석|
|문장 쌍 분류|문서(혹은 문장) 2개|두 문서가 속하는 범주에 대한 확률|자연어 추론|
|시퀀스 레이블링|문서(혹은 문장) 1개|토큰 각각의 범주 확률|개체명 인식|